In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/'CS685'/Project/
!ls

Mounted at /gdrive
/gdrive/MyDrive/CS685/Project
eval.ipynb				 output
Labels_gold_silver_model_4_one_shot.txt  Preds_gold_silver_model_4_one_shot.txt
models					 SNIPS
N_Shot_Learning_Model_2_SNIPS.ipynb	 test.csv
N_Shot_Learning_Model_3_SNIPS.ipynb	 train.csv
N_Shot_Lerning_Backup.ipynb		 val.csv


In [2]:
#!pip install transformers
!pip install evaluate
!pip install datasets
!pip install sacrebleu

!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [3]:
import torch 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, Seq2SeqTrainer, TrainingArguments, DataCollatorForSeq2Seq, AutoModelForSequenceClassification, Seq2SeqTrainingArguments
import numpy as np
import evaluate
from datasets import load_dataset, Dataset
import pandas as pd


In [4]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
!nvidia-smi

Thu May 18 01:56:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# model_name_or_path = "t5-small"
model_name_or_path = "output/without_intent/gold_silver_model_2"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

In [8]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [9]:
train_df = pd.read_csv("train.csv")
print('train set length')
print(len(train_df.index))
get_random_sample = lambda x: x.iloc[np.random.randint(0, len(x))]
shots_list = [] # stores df for 1,2,4,8,16 shot training data
prev = None
for i in range(16):
  dummy = train_df.groupby('label').apply(get_random_sample)
  curr = dummy.reset_index(drop = True)
  if i != 0:
    curr = prev.append(curr, ignore_index = True)
  prev = curr
  if i == 0 or i == 1 or i == 3 or i == 7 or i == 15:
    shots_list.append(curr)


one_shot_train_df = shots_list[0]
two_shot_train_df = shots_list[1]
four_shot_train_df = shots_list[2]
eight_shot_train_df = shots_list[3]
sixteen_shot_train_df = shots_list[4]
train_set_name = ['one_shot', 'two_shot', 'four_shot', 'eight_shot', 'sixteen_shot', 'full_resource']

all_shots = {}

all_shots['one_shot'] = one_shot_train_df
all_shots['two_shot'] = two_shot_train_df
all_shots['four_shot'] = four_shot_train_df
all_shots['eight_shot'] = eight_shot_train_df
all_shots['sixteen_shot'] = sixteen_shot_train_df
all_shots['full_resource'] = train_df


train set length
13084


<ipython-input-9-54c34ee3854d>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-9-54c34ee3854d>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-9-54c34ee3854d>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-9-54c34ee3854d>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-9-54c34ee3854d>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

In [10]:
data_files = {"test": "test.csv", "validation": "val.csv"}

raw_datasets = load_dataset("./", data_files = data_files)

print(raw_datasets)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/.-292a27d75e218004/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 700
    })
})


In [11]:
#metric = evaluate.load("sacrebleu")
#accuracy_metric = evaluate.load("accuracy")
prefix = "intent classification: "


# def compute_metrics(eval_pred):
#     # predictions, labels = eval_pred
#     # decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     # result = bleu.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     # result["gen_len"] = np.mean(prediction_lens)

#     # return {k: round(v, 4) for k, v in result.items()}

#         preds, labels = eval_pred
#         if isinstance(preds, tuple):
#             preds = preds[0]
#         decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#             # Replace -100 in the labels as we can't decode them.
#         labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#         decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#         # Some simple post-processing
#         decoded_preds = [pred.strip() for pred in decoded_preds]
#         decoded_labels = [label.strip() for label in decoded_labels]
#         decoded_labels = [[label] for label in decoded_labels]
#         result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#         prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#         result["gen_len"] = np.mean(prediction_lens)

#         return result

def get_decoded_labels_preds(preds, labels):
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    decoded_labels = [label for label in decoded_labels]

    return (decoded_preds, decoded_labels)

def compute_basic_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds, decoded_labels = get_decoded_labels_preds(preds,labels)
    
    equals = 0
    # print(decoded_preds)
    for i in range(len(decoded_labels)): 
      
      if decoded_labels[i] == decoded_preds[i]:
        equals += 1
    return {"accuracy": equals/len(decoded_labels)}


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["label"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [12]:

for index in range(len(train_set_name)):
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
  model.to(device)
  raw_datasets["train"] = Dataset.from_pandas(all_shots[train_set_name[index]])
  tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
  tokenized_datasets = tokenized_datasets.remove_columns(raw_datasets["train"].column_names)

  datacollator = DataCollatorForSeq2Seq(tokenizer = tokenizer, return_tensors= "pt", padding=True)
  training_args = Seq2SeqTrainingArguments(evaluation_strategy = "epoch", output_dir="./models",
                                  learning_rate= 1e-2,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=16,
                                  save_total_limit=3,
                                  num_train_epochs=10,
                                  predict_with_generate=True,
                                  fp16=True,
                                  logging_strategy = 'epoch')
  trainer = Seq2SeqTrainer(model = model, 
                  args = training_args, 
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"],
                  data_collator = datacollator,
                  tokenizer = tokenizer,
                  compute_metrics = compute_basic_metrics
                  )
  trainer.train()


  ## Evaluation on Test set.

  predictions, labels, metrics = trainer.predict(tokenized_datasets["test"])
  decoded_preds, decoded_labels = get_decoded_labels_preds(predictions,labels)
  print('For ' + model_name_or_path +  '_' + train_set_name[index] + ' metrics are: ')
  print(metrics)
  
  concat_preds = u'\n'.join(decoded_preds).encode('utf-8').strip()
  concat_labels = u'\n'.join(decoded_labels).encode('utf-8').strip()
  try:
    name_start_index = model_name_or_path.rindex('/')
  except: 
    name_start_index = -1
  file_name_preds = 'Preds_' + model_name_or_path[name_start_index + 1 : ] + "_" + train_set_name[index] + '.txt'
  file_name_labels = 'Labels_' + model_name_or_path[name_start_index + 1 : ] + "_" + train_set_name[index] + '.txt'

  print(file_name_preds)
  with open(file_name_preds, 'wb') as f:
    f.write(concat_preds)

  with open(file_name_labels, 'wb') as f:
    f.write(concat_labels)


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,11.889700,12.230122,0.000000
2,11.939700,12.230122,0.000000
3,11.940400,12.230122,0.000000
4,12.455300,12.230122,0.000000
5,11.852300,6.251159,0.000000
6,7.079600,3.155670,0.000000
7,2.736900,1.905815,0.012857
8,1.550100,1.136664,0.215714
9,0.753000,0.615038,0.432857
10,0.346700,0.445035,0.490000


...

Epoch,Training Loss,Validation Loss,Accuracy
1,10.406700,4.487027,0.004286
2,2.766300,0.452312,0.397143
3,0.317000,0.329041,0.574286
4,0.153700,0.087187,0.885714
5,0.023800,0.080255,0.925714
6,0.004400,0.074492,0.937143
7,0.002300,0.082814,0.915714
8,0.007500,0.105356,0.880000
9,0.005300,0.099435,0.898571
10,0.002900,0.082886,0.931429


For output/without_intent/gold_silver_model_2_sixteen_shot metrics are: 
{'test_loss': 0.0509980134665966, 'test_accuracy': 0.9485714285714286, 'test_runtime': 3.6129, 'test_samples_per_second': 193.75, 'test_steps_per_second': 12.179}
Preds_gold_silver_model_2_sixteen_shot.txt


Map:   0%|          | 0/13084 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.176000,0.048950,0.948571
2,0.066200,0.061447,0.901429
3,0.067900,0.039139,0.955714
4,0.052900,0.040109,0.945714
5,0.040600,0.045762,0.958571
6,0.032600,0.035052,0.960000
7,0.025400,0.027045,0.968571
8,0.021200,0.034507,0.967143
9,0.018600,0.025631,0.965714
10,0.016800,0.023970,0.971429


For output/without_intent/gold_silver_model_2_full_resource metrics are: 
{'test_loss': 0.029321182519197464, 'test_accuracy': 0.9671428571428572, 'test_runtime': 3.8967, 'test_samples_per_second': 179.638, 'test_steps_per_second': 11.292}
Preds_gold_silver_model_2_full_resource.txt
